# Advanced matplotlib

Pierre Augier (LEGI), Cyrille Bonamy (LEGI), Eric Maldonado (Irstea), Franck Thollard (ISTerre), Christophe Picard (LJK), Loïc Huder (ISTerre)

## Introduction
This is the second part of the introductive presentation given in the [Python initiation training](https://gricad-gitlab.univ-grenoble-alpes.fr/python-uga/py-training-2017/blob/master/ipynb/pres111_intro_matplotlib.ipynb). 

The aim is to present more advanced usecases of matplotlib.

## Quick reminders

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Object-oriented plots

In matplotlib plots, **everything** is an object. It is therefore possible to change every aspects of the figure by acting on the appropriate objects.

### Subplots (gridspec)

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure()
gs = gridspec.GridSpec(2, 2, figure=fig) # 2 rows and 2 columns
X = np.arange(-3, 3, 0.01)*np.pi
ax1 = fig.add_subplot(gs[0,0]) # 1st row, 1st column
ax2 = fig.add_subplot(gs[1,0]) # 2nd row, 1st column
ax3 = fig.add_subplot(gs[:,1]) # all rows, 2nd column
ax1.plot(X, np.cos(2*X), color="red")
ax2.plot(X, np.sin(2*X), color="magenta")
ax3.plot(X, X**2)

## Interactivity

## Animations

From the matplotlib page (https://matplotlib.org/api/animation_api.html):
> The easiest way to make a live animation in matplotlib is to use one of the Animation classes.
><table>
    <tr><td>FuncAnimation</td><td>Makes an animation by repeatedly calling a function func.</td></tr>
    <tr><td>ArtistAnimation</td><td>Animation using a fixed set of Artist objects.</td></tr>
</table>

### Example from matplotlib page
This example uses `FuncAnimation` to animate the plot of a sin function.

The animation consists in making repeated calls to the `update` function that adds at each frame a datapoint to the plot.

In [ ]:
from matplotlib import animation

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'ro')

def init():
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

ani = animation.FuncAnimation(fig, update, frames=np.linspace(0, 2*np.pi, 128),
                    init_func=init, blit=True)
plt.show()


The previous code executed in a regular Python script should display the animation without problem. In a Jupyter Notebook, it is necessary to use IPython to display it in HTML.

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

### Stroop test
The [Stroop effect](http://en.wikipedia.org/wiki/Stroop_effect) is when a psychological cause inteferes with the reaction time of a task.

A common demonstration of this effect (called a Stroop test) is reading a word decribing a color that is written in another color. This usually takes longer than reading a non-colored word.

Ex: <div style='text-align:center; font-size:36px'><span style='color:blue'>RED</span> vs. <span style='color:blue'>BIRD</span></div>

_Funfact: As this test relies on the significance of the words, people that are more used to English should find the test more difficult !_

In this part, we show how `matplotlib` animations can generate a Stroop test that shows random colour words in random colors at random positions.

#### With `FuncAnimation`
We will generate a single object `word` whose position, color and text will be updated by the repeatedly called function.

In [ ]:
import random

def generate_random_colored_word(words, colors):
    displayed_text = random.choice(words).upper()
    text_color = random.choice(colors)
    xy_position = (random.random(), random.random())
    return xy_position, displayed_text, text_color


def update(frame):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    word.set_position(xy_position)
    word.set_color(text_color)
    word.set_text(displayed_text)
    return word

fig, ax = plt.subplots()

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset

xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
word = ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36)

ani = animation.FuncAnimation(fig, update, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

#### With `ArtistAnimation`
Rather than updating through a function, `ArtistAnimation` requires to generate first all the `Artists` that will be displayed during the whole animation. 

A list of `Artists` must therefore be supplied for each frame. Then, all frame lists must be compiled in a single list (of lists) that will be given in argument of `ArtistAnimation`.

In our case, to reproduce the behaviour above, we need to have only one word per frame. Each frame will therefore have a list of a single element (the colored word for this frame).

In [ ]:
fig, ax = plt.subplots()

N_frames = 200
words = []

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset

# Generate the list of lists of Artists.
for i in range(N_frames):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    # The list of the frame contains only a single word
    frame_artists = [ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36)]
    words.append(frame_artists)

ani = animation.ArtistAnimation(fig, words, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())

#### Example with multiple `Artists`: two words at once from two wordsets !

In [ ]:
# We can remove the axes for a cleaner test
fig = plt.figure()
ax = fig.add_subplot(111, frameon=False)
ax.set_xticks([])
ax.set_yticks([])

N_frames = 200
words = []

colorset = ['red', 'blue', 'yellow', 'green', 'purple']
wordset = colorset
wordset2 = ['bed', 'glue', 'mellow', 'grain', 'people']

# Generate the list of lists of Artists.
for i in range(N_frames):
    xy_position, displayed_text, text_color = generate_random_colored_word(wordset, colorset)
    xy_position2, displayed_text2, text_color2 = generate_random_colored_word(wordset2, colorset)
    # The list of the frame contains only a single word
    frame_artists = [ax.annotate(displayed_text, xy_position, xycoords='axes fraction', color=text_color, size=36),
                     ax.annotate(displayed_text2, xy_position2, xycoords='axes fraction', color=text_color2, size=36)]
    words.append(frame_artists)

ani = animation.ArtistAnimation(fig, words, interval=1000)
plt.show()

In [ ]:
from IPython.display import HTML

HTML(ani.to_jshtml())